# Implementing a Plan-and-Execute Agent with Router

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['OPEN OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') # type: ignore

In [10]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

## 1 - Define the State

Track the plan, past steps, response, and original input

In [11]:
class State(TypedDict):
    """Normal state"""
    messages: Annotated[list, add_messages]

## 3 - Define the memory to persist the conversations

**NOTE**: Use SqliteSaver or PostgresSaver to persiste conversation rather than using InMemorySaver.

In [12]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

## 4 - Create a simple chat graph


In [8]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1")

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

## Adding memory to the graph
graph = graph_builder.compile(checkpointer=memory)

## 5 - Testing the memory of the chat

In [9]:
config = { "configurable": { "thread_id": "1" } }

events = graph.stream(
    { "messages": [{ "role": "user", "content": "Hola soy Diego!!" }]},
    config, # type: ignore
    stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hola soy Diego!!
================================== Ai Message ==================================

¡Hola, Diego! 😊 ¿Cómo estás? ¿En qué puedo ayudarte hoy?


In [13]:
config = { "configurable": { "thread_id": "1" } }

events = graph.stream(
    { "messages": [{ "role": "user", "content": "Cual te dije que era mi nombre?" }]},
    config, # type: ignore
    stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Cual te dije que era mi nombre?
================================== Ai Message ==================================

Me dijiste que tu nombre es **Diego**. ¿Acerté? 😊


## 6 - Testing with other user

In [14]:
config = { "configurable": { "thread_id": "2" } }

events = graph.stream(
    { "messages": [{ "role": "user", "content": "Cual te dije que era mi nombre?" }]},
    config, # type: ignore
    stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Cual te dije que era mi nombre?
================================== Ai Message ==================================

No me has dicho tu nombre aún. ¿Quieres decírmelo ahora?
